# Load segmentation, localise and save as h5

Segment a stack of images and then manually label a couple, then check how well the model segmented them.

In [1]:
import napari
import cellpose
from octopuslite import utils, tile
import numpy as np

import sys
sys.path.append('macrohet/')
from notify import send_sms

def view(img):
    return napari.Viewer().add_image(img)

from tqdm.auto import tqdm

import btrack
import dask.array as da

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
utils.read_harmony_metadata(metadata_path, assay_layout=True)

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [4]:
row = '6'
column = '9'

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [5]:
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, 
                             column, 
                             #set_channel=1, 
                             set_plane = 'sum_proj',
#                              set_time = 1
                         )#.astype(uint8)

#### Reorder the channel axis for localisation

In [6]:
gfp = images[:,0,...]
rfp = images[:,1,...]
images = da.stack([gfp,rfp], axis = -1)
images

,Array,Chunk
Bytes,40.88 GiB,31.01 MiB
Shape,"(75, 6048, 6048, 2)","(1, 2016, 2016, 1)"
Count,18900 Tasks,1350 Chunks
Type,uint64,numpy.ndarray


### Load masks

In [9]:
masks = np.load('/mnt/DATA/macrohet/segmentation/[6,9]_masks.npy')

In [13]:
masks = da.asarray(masks)

In [14]:
masks

dask.array<array, shape=(75, 6048, 6048), dtype=uint16, chunksize=(75, 864, 864), chunktype=numpy.ndarray>

# Localise raw masks

Include size filter and measurements of fluorescence

In [15]:
feat = [
      "area",
      "major_axis_length",
      "minor_axis_length",
      "orientation",
      "mean_intensity",
        ]

In [16]:
objects = btrack.utils.segmentation_to_objects(
    masks, 
    images,
    properties = tuple(feat),
    use_weighted_centroid = False, 
)

[INFO][2023/01/24 11:00:14 AM] Localizing objects from segmentation...
[INFO][2023/01/24 11:00:14 AM] Found intensity_image data
[INFO][2023/01/24 11:17:52 AM] Objects are of type: <class 'dict'>
[INFO][2023/01/24 11:17:52 AM] ...Found 89235 objects in 75 frames.


In [17]:
objects = [o for o in objects if o.properties['area'] > 2500]

In [18]:
with btrack.dataio.HDF5FileHandler(
     '/mnt/DATA/macrohet/segmentation/[6,9]_masks.h5', 'w', obj_type='obj_type_1',
) as hdf:
    hdf.write_segmentation(masks)
    hdf.write_objects(objects)

[INFO][2023/01/24 11:17:53 AM] Opening HDF file: /mnt/DATA/macrohet/segmentation/[6,9]_masks.h5...
[INFO][2023/01/24 11:18:32 AM] Writing objects/obj_type_1
[INFO][2023/01/24 11:18:32 AM] Writing labels/obj_type_1
[INFO][2023/01/24 11:18:32 AM] Loading objects/obj_type_1 (40381, 5) (40381 filtered: None)
[INFO][2023/01/24 11:18:33 AM] Writing properties/obj_type_1/area (40381,)
[INFO][2023/01/24 11:18:33 AM] Writing properties/obj_type_1/major_axis_length (40381,)
[INFO][2023/01/24 11:18:33 AM] Writing properties/obj_type_1/minor_axis_length (40381,)
[INFO][2023/01/24 11:18:33 AM] Writing properties/obj_type_1/orientation (40381,)
[INFO][2023/01/24 11:18:33 AM] Writing properties/obj_type_1/mean_intensity-0 (40381,)
[INFO][2023/01/24 11:18:33 AM] Writing properties/obj_type_1/mean_intensity-1 (40381,)
[INFO][2023/01/24 11:18:33 AM] Closing HDF file: /mnt/DATA/macrohet/segmentation/[6,9]_masks.h5


In [19]:
objects[0]

,ID,x,y,z,t,dummy,states,label,area,major_axis_length,minor_axis_length,orientation,mean_intensity-0,mean_intensity-1
0,0,257.838564,104.539205,0.0,0,False,7,5,36194,244.63143,204.217784,-0.625041,1751.473891,343.588302


# Load h5

In [22]:
with btrack.dataio.HDF5FileHandler(
     '/mnt/DATA/macrohet/segmentation/[6,9]_masks.h5', 'r', obj_type='obj_type_1',
) as hdf:
    loaded_masks = hdf.segmentation
    loaded_objects = hdf.objects

[INFO][2023/01/24 02:04:47 pm] Opening HDF file: /mnt/DATA/macrohet/segmentation/[6,9]_masks.h5...
24-Jan-23 14:04:47 - btrack.dataio - INFO     - Opening HDF file: /mnt/DATA/macrohet/segmentation/[6,9]_masks.h5...
[INFO][2023/01/24 02:05:00 pm] Loading segmentation (75, 6048, 6048)
24-Jan-23 14:05:00 - btrack.dataio - INFO     - Loading segmentation (75, 6048, 6048)
[INFO][2023/01/24 02:05:00 pm] Loading objects/obj_type_1 (40381, 5) (40381 filtered: None)
24-Jan-23 14:05:00 - btrack.dataio - INFO     - Loading objects/obj_type_1 (40381, 5) (40381 filtered: None)
[INFO][2023/01/24 02:05:01 pm] Closing HDF file: /mnt/DATA/macrohet/segmentation/[6,9]_masks.h5
24-Jan-23 14:05:01 - btrack.dataio - INFO     - Closing HDF file: /mnt/DATA/macrohet/segmentation/[6,9]_masks.h5


In [24]:
loaded_objects[0]

,ID,x,y,z,t,dummy,states,label,area,major_axis_length,mean_intensity-0,mean_intensity-1,minor_axis_length,orientation
0,0,257.838562,104.539207,0.0,0,False,7,5,36194.0,244.631424,1751.473877,343.588287,204.217789,-0.625041


In [43]:
objects = loaded_objects

# Track

In [37]:
from btrack import datasets

In [40]:
config = '/home/dayn/analysis/BayesianTracker/models/particle_config.json'

In [55]:
features = list(objects[0].properties.keys())

In [56]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure(config)
    tracker.max_search_radius = 250
    tracker.tracking_updates = ["MOTION", "VISUAL"]
    tracker.features = features

    # append the objects to be tracked
    tracker.append(objects)

    # set the tracking volume
    tracker.volume=((0, masks.shape[-2]), (0, masks.shape[-1]))

    # track them (in interactive mode)
    tracker.track(step_size=5)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari()
    
    # store the tracks
    tracks = tracker.tracks
    
    # store the configuration
    cfg = tracker.configuration
    
    # export the track data 
    tracker.export("/mnt/DATA/macrohet/segmentation/[6,9]_tracks.h5", obj_type="obj_type_1")
    

[INFO][2023/01/24 02:40:00 pm] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
24-Jan-23 14:40:00 - btrack.libwrapper - INFO     - Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2023/01/24 02:40:00 pm] btrack (v0.5.0) library imported
24-Jan-23 14:40:00 - btrack.core - INFO     - btrack (v0.5.0) library imported
[INFO][2023/01/24 02:40:00 pm] Starting BayesianTracker session
24-Jan-23 14:40:00 - btrack.core - INFO     - Starting BayesianTracker session
[INFO][2023/01/24 02:40:00 pm] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/particle_config.json
24-Jan-23 14:40:00 - btrack.config - INFO     - Loading configuration file: /home/dayn/analysis/BayesianTracker/models/particle_config.json
[INFO][2023/01/24 02:40:00 pm] Objects are of type: <class 'list'>
24-Jan-23 14:40:00 - btrack.dataio - INFO     - Objects are of type: <class 'list'>
[INFO][2023/01/24 02:40:01 pm] Starting tracking... 
24-Jan-23 

[INFO][2023/01/24 02:40:50 pm]  - Probabilities (Link: 1.00000, Lost: 1.00000)
24-Jan-23 14:40:50 - btrack.utils - INFO     -  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2023/01/24 02:40:50 pm]  - Stats (Active: 1269, Lost: 60562, Conflicts resolved: 10529)
24-Jan-23 14:40:50 - btrack.utils - INFO     -  - Stats (Active: 1269, Lost: 60562, Conflicts resolved: 10529)
[INFO][2023/01/24 02:40:50 pm] Tracking objects in frames 50 to 54 (of 75)...
24-Jan-23 14:40:50 - btrack.core - INFO     - Tracking objects in frames 50 to 54 (of 75)...
[INFO][2023/01/24 02:40:52 pm]  - Timing (Bayesian updates: 85.65ms, Linking: 4.78ms)
24-Jan-23 14:40:52 - btrack.utils - INFO     -  - Timing (Bayesian updates: 85.65ms, Linking: 4.78ms)
[INFO][2023/01/24 02:40:52 pm]  - Probabilities (Link: 1.00000, Lost: 1.00000)
24-Jan-23 14:40:52 - btrack.utils - INFO     -  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2023/01/24 02:40:52 pm]  - Stats (Active: 1083, Lost: 63772, Conflicts resolved:

GLPK Integer Optimizer 5.0
40096 rows, 32487 columns, 44926 non-zeros
32487 integer variables, all of which are binary
Preprocessing...
20048 rows, 32487 columns, 44926 non-zeros
32487 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 20048
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
20048 rows, 32487 columns, 44926 non-zeros
*     0: obj =   1.170319066e+05 inf =   0.000e+00 (6696)
Perturbing LP to avoid stalling [2507]...
*  6268: obj =   7.381611874e+04 inf =   0.000e+00 (434)
Removing LP perturbation [6716]...
*  6716: obj =   7.344274620e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  6716: mip =     not found yet >=              -inf        (1; 0)
+  6716: >>>>>   7.344274620e+04 >=   7.344274620e+04   0.0% (1; 0)
+  6716: mip =   7.344

[INFO][2023/01/24 02:41:03 pm] Opening HDF file: /mnt/DATA/macrohet/segmentation/[6,9]_tracks.h5...
24-Jan-23 14:41:03 - btrack.dataio - INFO     - Opening HDF file: /mnt/DATA/macrohet/segmentation/[6,9]_tracks.h5...
[INFO][2023/01/24 02:41:04 pm] Writing objects/obj_type_1
24-Jan-23 14:41:04 - btrack.dataio - INFO     - Writing objects/obj_type_1
[INFO][2023/01/24 02:41:04 pm] Writing labels/obj_type_1
24-Jan-23 14:41:04 - btrack.dataio - INFO     - Writing labels/obj_type_1
[INFO][2023/01/24 02:41:04 pm] Loading objects/obj_type_1 (40381, 5) (40381 filtered: None)
24-Jan-23 14:41:04 - btrack.dataio - INFO     - Loading objects/obj_type_1 (40381, 5) (40381 filtered: None)
[INFO][2023/01/24 02:41:04 pm] Writing properties/obj_type_1/area (40381,)
24-Jan-23 14:41:04 - btrack.dataio - INFO     - Writing properties/obj_type_1/area (40381,)
[INFO][2023/01/24 02:41:04 pm] Writing properties/obj_type_1/major_axis_length (40381,)
24-Jan-23 14:41:04 - btrack.dataio - INFO     - Writing propert

In [59]:
images

,Array,Chunk
Bytes,40.88 GiB,31.01 MiB
Shape,"(75, 6048, 6048, 2)","(1, 2016, 2016, 1)"
Count,18900 Tasks,1350 Chunks
Type,uint64,numpy.ndarray


In [60]:
masks

dask.array<array, shape=(75, 6048, 6048), dtype=uint16, chunksize=(75, 864, 864), chunktype=numpy.ndarray>

In [61]:
v = napari.Viewer()
v.add_image(images, 
            channel_axis = -1, 
            colormap=['green', 'magenta'], 
            contrast_limits=[[100,6000],[0,3000]])
# for key in masks_dict.keys():
#     v.add_labels(masks_dict[key][0], name = key)
v.add_labels(masks, name = 'masks')
v.add_tracks(
                    data, 
                    properties=properties, 
                    graph=graph, 
                    name="Tracks", 
                    blending="translucent",
                    visible=False,
                )


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


NameError: name 'viewer' is not defined